In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas_datareader import data
from scipy.optimize import differential_evolution, shgo, NonlinearConstraint
import sys

sys.path.append('..')

#from src import data
#from src.portfolio import Portfolio, fees_func

In [37]:
def quotes(assets, look_back, end_date=None):
    if end_date:
        start_date = (end_date - timedelta(days=look_back)).strftime("%Y-%m-%d")
    else:
        start_date = (datetime.today() - timedelta(days=look_back)).strftime("%Y-%m-%d")

    df = data.DataReader(assets, "yahoo", start=start_date, end=end_date)
    df = df.stack(level=1).reset_index(level=[0, 1], drop=False).reset_index(drop=True)
    df = df.sort_values(["Symbols", "Date"])
    
    return df


df_portfolio = pd.DataFrame({
    "Assets": ["CW8.PA", "PAEEM.PA", "RS2K.PA", "SMC.PA", "EESM.PA"],
    "Weights": [.84, .04, .06, .03, .03],
    "Shares": [53, 51, 4, 1, 2],
})

assets = df_portfolio["Assets"].values.ravel()
shares = df_portfolio["Shares"].values.ravel()
target_weights = df_portfolio["Weights"].values.ravel()

df_prices = quotes(assets, 30)

In [50]:
last_trading_day = df_prices["Date"].max()
df_prices.loc[df_prices["Date"] == last_trading_day, "Close"]

110    397.835602
114    312.954987
111     23.441000
112    269.951294
113    324.100006
Name: Close, dtype: float64

In [53]:
df_prices.loc[df_prices["Date"] == last_trading_day, "Close"].values

array([397.83560181, 312.95498657,  23.44099998, 269.95129395,
       324.1000061 ])

In [2]:
assets = ['CW8.PA', 'PAEEM.PA', 'RS2K.PA', 'SMC.PA', 'EESM.PA']

# inputs
target_weights = np.array([.84, .04, .06, .03, .03])
shares = np.array([53, 51, 4, 1, 2])
cash = 1538.95

# current
prices = data.daily_close(assets, 0).values.ravel()
positions = prices * shares
portfolio_value = positions.sum() + cash
weights = positions / portfolio_value
target_positions = portfolio_value * target_weights

In [11]:
portfolio = Portfolio(shares, cash, fees_func)
shares_buy = portfolio.rebalance(prices, target_weights)

transaction = shares_buy * prices
fees = np.array([fees_func(x) for x in shares_buy * prices])
cash_leftover = cash - (transaction.sum() + fees.sum())

print(portfolio.results)
print(f"Shares to buy: {shares_buy}")
print(f"Transaction: {transaction.sum():.2f}€")
print(f"Fees: {fees.sum():.2f}€")
print(f"Cash leftover: {cash_leftover:.2f}€")

           constr: [array([0.])]
 constr_violation: 0.0
              fun: 473.15652976876277
            maxcv: 0.0
          message: 'Optimization terminated successfully.'
             nfev: 3305
              nit: 56
          success: True
                x: array([1.38236333, 0.43445608, 2.9511558 , 1.17720513, 0.01353664])
Shares to buy: [1. 0. 3. 1. 0.]
Transaction: 1521.53€
Fees: 12.50€
Cash leftover: 4.92€


In [13]:
positions_new = (shares + shares_buy) * prices
new_weights = positions_new / portfolio_value

print(f"New weights: {new_weights}")
print(f"Target: {target_weights}")
print(f"Old weights: {weights}")
print(f"RMSE: {np.sqrt(((weights - new_weights) ** 2).sum()):.3f}")

New weights: [0.83163396 0.04561493 0.07302474 0.0249225  0.02412669]
Target: [0.84 0.04 0.06 0.03 0.03]
Old weights: [0.81623333 0.04561493 0.04172842 0.01246125 0.02412669]
RMSE: 0.037


In [29]:
pd.DataFrame(data=[new_weights], columns=assets).style.format(lambda x: f"{x:.2%}")

,CW8.PA,PAEEM.PA,RS2K.PA,SMC.PA,EESM.PA
0,83.16%,4.56%,7.30%,2.49%,2.41%
